Ref: [notebook](https://colab.research.google.com/github/huggingface/workshops/blob/main/luzern-university/02-text-classification.ipynb)

Dataset **amazon_reviews_multi** is defunct and no longer accessible due to the decision of data providers. We are using **xonic48/amazon_review_multi** dataset instead.


## Setup

In [1]:
! pip install datasets transformers sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


## Prepare Dataset

### Load Dataset

In [5]:
# check the languages reviews are avaialable in
from datasets import get_dataset_config_names
dataset_name = "xonic48/amazon_review_multi" #"amazon_reviews_multi": Dataset 'amazon_reviews_multi' is defunct and no longer accessible due to the decision of data providers
langs = get_dataset_config_names(dataset_name)
print("langs= ",langs)

# load english amazon reviews
from datasets import load_dataset
marc = load_dataset(dataset_name)
marc_en = marc.filter(lambda example: example["language"]=="en")
#marc_en = load_dataset(path=dataset_name, name="en") #load_dataset() will cache the files at ~/.cache/huggingface/dataset/, so you won't need to re-download the dataset the next time your run the notebook.
print(marc_en)

langs=  ['default']


train.csv:   0%|          | 0.00/346M [00:00<?, ?B/s]

archive/validation.csv:   0%|          | 0.00/8.55M [00:00<?, ?B/s]

archive/test.csv:   0%|          | 0.00/8.60M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1200000 [00:00<?, ? examples/s]

...

Generating test split:   0%|          | 0/30000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1200000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/30000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/30000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 200000
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
    test: Dataset({
        features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 5000
    })
})


### Peek at Dataset

Get a rough idea about the kind of data we're dealing with

In [6]:
# Peek at first element
print(marc_en["train"][:3]) # we will get a list of values for each column. This is because Hugging Face Datasets is based on Apache Arrow, which defines a typed columnar format that is very memory efficient.

# display a nicer for a random sample
from IPython.display import display, HTML
marc_en.set_format("pandas")
df = marc_en["train"][:]
sample = df.sample(n=5, random_state=42)
display(HTML(sample.to_html()))

{'Unnamed: 0': [200000, 200001, 200002], 'review_id': ['en_0964290', 'en_0690095', 'en_0311558'], 'product_id': ['product_en_0740675', 'product_en_0440378', 'product_en_0399702'], 'reviewer_id': ['reviewer_en_0342986', 'reviewer_en_0133349', 'reviewer_en_0152034'], 'stars': [1, 1, 1], 'review_body': ["Arrived broken. Manufacturer defect. Two of the legs of the base were not completely formed, so there was no way to insert the casters. I unpackaged the entire chair and hardware before noticing this. So, I'll spend twice the amount of time boxing up the whole useless thing and send it back with a 1-star review of part of a chair I never got to sit in. I will go so far as to include a picture of what their injection molding and quality assurance process missed though. I will be hesitant to buy again. It makes me wonder if there aren't missing structures and supports that don't impede the assembly process.", 'the cabinet dot were all detached from backing... got me', "I received my first o

,Unnamed: 0,review_id,product_id,reviewer_id,stars,review_body,review_title,language,product_category
119737,319737,en_0522546,product_en_0681589,reviewer_en_0687817,3,"Not strong enough to run a small 120v vacuum cleaner, to clean car.",Not strong enough to run a small 120v vacuum cleaner ...,en,lawn_and_garden
72272,272272,en_0612910,product_en_0295449,reviewer_en_0312138,2,"The leg openings are a little small, but other than that the suit fits nicely, and is high quality material. Edit: I have been wearing this for less than two months and it is 100% worn out. It has worn so thin in multiple spots that it’s no longer appropriate for wearing in public, I have to throw it away. This is unacceptable.",Crap,en,apparel
158154,358154,en_0983065,product_en_0295095,reviewer_en_0927618,4,Really cute mug. I would have given 5 stars if it were a bit bigger.,Four Stars,en,kitchen
65426,265426,en_0206761,product_en_0563487,reviewer_en_0936741,2,Well it’s looks and feels okay but it most certainly does not have 4 pockets that’s a lie it has 3 so that’s pretty messed up to say it has 4 when it’s only 3 the fabric is super stiff hopefully after washing it will be better,Lies!!,en,industrial_supplies
30074,230074,en_0510474,product_en_0704805,reviewer_en_0417600,1,"Very, very thin, you can bend them with you fingers with no problem! Print is small.. More of a decoration. Would give 1/2 star!",Thin and bendable :(,en,pet_products


In [7]:
df["product_category"].value_counts()

,count
product_category,
home,17679
apparel,15951
wireless,15717
other,13418
beauty,12091
drugstore,11730
kitchen,10382
toy,8745
sports,8277


In [8]:
df["stars"].value_counts()

,count
stars,
1,40000
2,40000
3,40000
4,40000
5,40000


Now that we've got a rough idea about the kind of data we're dealing with, let's reset the output format from pandas back to arrow:

In [9]:
marc_en.reset_format()

### Filtering for a product category

Although we could go ahead and fine-tune a Transformer model on the whole set of 200,000 English reviews, this will take several hours on a single GPU. So instead, we'll focus on fine-tuning a model for a single product category.

In [10]:
product_category = "book"

def filter_for_product(example, product_category=product_category):
    return example["product_category"] == product_category

product_dataset = marc_en.filter(filter_for_product)
print(product_dataset)

Filter:   0%|          | 0/200000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 3756
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 82
    })
    test: Dataset({
        features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category'],
        num_rows: 105
    })
})


### Mapping the labels

During training, 🤗 **Transformers expects the labels to be ordered, starting from 0 to N**. But we've seen that `our star ratings range from 1-5`, so let's fix that. While we're at it, we'll create a mapping between the label IDs and names, which will be handy later on when we want to run inference with our model. First we'll define the label mapping from ID to name:

In [11]:
label_names = ["terrible", "poor", "ok", "good", "great"]
id2label = {idx:label for idx, label in enumerate(label_names)}
print(id2label)

{0: 'terrible', 1: 'poor', 2: 'ok', 3: 'good', 4: 'great'}


In [12]:
#id to label mapping
def map_labels(example):
    label_id = example["stars"] - 1 # Shift labels to start from 0
    return {"labels": label_id, "label_name": id2label[label_id]}

#label to id mapping
label2id = {v:k for k,v in id2label.items()}


product_dataset = product_dataset.map(map_labels)
product_dataset["train"][0]

Map:   0%|          | 0/3756 [00:00<?, ? examples/s]

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

Map:   0%|          | 0/105 [00:00<?, ? examples/s]

{'Unnamed: 0': 200031,
 'review_id': 'en_0369494',
 'product_id': 'product_en_0447174',
 'reviewer_id': 'reviewer_en_0879303',
 'stars': 1,
 'review_body': 'I ordered this book on February 11. It never arrived.',
 'review_title': 'Failure to send book',
 'language': 'en',
 'product_category': 'book',
 'labels': 0,
 'label_name': 'terrible'}

### Tokenize dataset
Transformers expect their inputs in the form of numbers (not strings) and so some form of preprocessing is required. For NLP, this preprocessing step is called tokenization. Tokenization converts strings into atomic chunks called tokens, and these tokens are subsequently encoded as numerical vectors.

In [14]:
from transformers import AutoTokenizer

model_checkpoint = "xlm-roberta-base" #pretrained model
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

def tokenize_reviews(examples):
    return tokenizer(examples["review_body"], truncation=True, max_length=180)

tokenized_dataset = product_dataset.map(tokenize_reviews, batched=True)
print("tokenized_dataset=",tokenized_dataset)
print("tokenized_dataset['train'][0]=",tokenized_dataset["train"][0])

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/615 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Map:   0%|          | 0/3756 [00:00<?, ? examples/s]

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

Map:   0%|          | 0/105 [00:00<?, ? examples/s]

tokenized_dataset= DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'labels', 'label_name', 'input_ids', 'attention_mask'],
        num_rows: 3756
    })
    validation: Dataset({
        features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'labels', 'label_name', 'input_ids', 'attention_mask'],
        num_rows: 82
    })
    test: Dataset({
        features: ['Unnamed: 0', 'review_id', 'product_id', 'reviewer_id', 'stars', 'review_body', 'review_title', 'language', 'product_category', 'labels', 'label_name', 'input_ids', 'attention_mask'],
        num_rows: 105
    })
})
tokenized_dataset['train'][0]= {'Unnamed: 0': 200031, 'review_id': 'en_0369494', 'product_id': 'product_en_0447174', 'reviewer_id': 'reviewer_en_0879303', 'stars': 1, 'review_body': 'I ordered this boo

Peek at tokenizer:

In [15]:
print("tokenizer.vocab_size=",tokenizer.vocab_size)
print("tokenizer.special_tokens_map=\n", tokenizer.special_tokens_map)

str= "Today I'm giving an NLP workshop at MLT"
encoded_str = tokenizer(str)
print("input string=", str)
print("tokenized/encoded string=",encoded_str)

for token in encoded_str["input_ids"]:
    print(token, tokenizer.decode([token]))

tokenizer.vocab_size= 250002
tokenizer.special_tokens_map=
 {'bos_token': '<s>', 'eos_token': '</s>', 'unk_token': '<unk>', 'sep_token': '</s>', 'pad_token': '<pad>', 'cls_token': '<s>', 'mask_token': '<mask>'}
input string= Today I'm giving an NLP workshop at MLT
tokenized/encoded string= {'input_ids': [0, 38396, 87, 25, 39, 68772, 142, 541, 37352, 42819, 99, 276, 27026, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
0 <s>
38396 Today
87 I
25 '
39 m
68772 giving
142 an
541 N
37352 LP
42819 workshop
99 at
276 M
27026 LT
2 </s>


## Loading a pretrained model

In [16]:
from transformers import AutoModelForSequenceClassification

num_labels = 5 #we have a 1-5 star classification system
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels, label2id=label2id, id2label=id2label)

model.safetensors:   0%|          | 0.00/1.12G [00:00<?, ?B/s]

Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Fine Tuning

Fine tuning pretrained model with product dataset.

### Create Trainer

In [17]:
from transformers import TrainingArguments

model_name = model_checkpoint.split("/")[-1]
batch_size = 16
num_train_epochs = 2
logging_steps = len(tokenized_dataset["train"]) // (batch_size * num_train_epochs)

# 1. Define hyperparameters
args = TrainingArguments(
    output_dir=f"{model_name}-finetuned-marc-en",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_train_epochs,
    weight_decay=0.01,
    logging_steps=logging_steps,
    push_to_hub=True,               #push each checkpoint to the Hub automatically for us, so we can reuse the model at any point in the future
)

# 2. Define evaluation metrics
#MARC paper authors recommend using mean absolute error (MAE) for star ratings because:
#star ratings for each review are ordinal, and a 2-star prediction for a 5-star review should be penalized more heavily than a 4-star prediction for a 5-star review.
import numpy as np
from sklearn.metrics import mean_absolute_error

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return {"MAE": mean_absolute_error(labels, predictions)}


# 3. Creae Trainer
from transformers import Trainer

trainer = Trainer(
    model,
    args, # train arguments
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics # evaluation metrics
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-17-c0cb6f9331a7>:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


### Train

In [18]:
 trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss,Mae
1,1.131100,1.050301,0.524390
2,1.008500,0.903525,0.475610


TrainOutput(global_step=470, training_loss=1.145409143985586, metrics={'train_runtime': 340.1173, 'train_samples_per_second': 22.086, 'train_steps_per_second': 1.382, 'total_flos': 513195491992128.0, 'train_loss': 1.145409143985586, 'epoch': 2.0})

In [19]:
trainer.push_to_hub(commit_message="Training complete!")

events.out.tfevents.1734545544.fa0a170930d1.220.0:   0%|          | 0.00/7.18k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/dmrhimali/xlm-roberta-base-finetuned-marc-en/commit/99e413be90c18b9808de9422c10a2e86025ed5ba', commit_message='Training complete!', commit_description='', oid='99e413be90c18b9808de9422c10a2e86025ed5ba', pr_url=None, repo_url=RepoUrl('https://huggingface.co/dmrhimali/xlm-roberta-base-finetuned-marc-en', endpoint='https://huggingface.co', repo_type='model', repo_id='dmrhimali/xlm-roberta-base-finetuned-marc-en'), pr_revision=None, pr_num=None)

### Evaluating cross-lingual transfer

Now that we're fine-tuned our model on a English subset, we can evaluate its ability to transfer to other languages. To do so, we'll load the validation set in a given language, apply the same filtering and preprocessing that we did for the English subset, and finally use `Trainer.evaluate()` to compute the metrics. The following function does the trick:

In [22]:
def evaluate_corpus(lang):
    # Load the language subset
    dataset = load_dataset(dataset_name, split="validation")
    dataset = dataset.filter(lambda example: example["language"]==lang)
    # Filter for the `sports` product category
    product_dataset = dataset.filter(filter_for_product)
    # Map and create label columns
    product_dataset = product_dataset.map(map_labels)
    # Tokenize the inputs
    tokenized_dataset = product_dataset.map(tokenize_reviews, batched=True)
    # Generate predictions and metrics
    preds = trainer.evaluate(eval_dataset=tokenized_dataset)
    return {"MAE": preds["eval_MAE"]}



Let's start with English (for reference our MAE on English was around 0.5): Note here our fine-tuned English model is able to transfer to English at roughly the same performance. Not bad!:

In [23]:
evaluate_corpus("en")

Filter:   0%|          | 0/30000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

Map:   0%|          | 0/82 [00:00<?, ? examples/s]

{'MAE': 0.47560975609756095}

In [24]:
evaluate_corpus("fr")

Filter:   0%|          | 0/30000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/215 [00:00<?, ? examples/s]

Map:   0%|          | 0/215 [00:00<?, ? examples/s]

{'MAE': 0.627906976744186}

In [25]:
evaluate_corpus("ja")

Filter:   0%|          | 0/30000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/121 [00:00<?, ? examples/s]

Map:   0%|          | 0/121 [00:00<?, ? examples/s]

{'MAE': 0.628099173553719}

Nice, Above languages mae is very similar too! This shows the great power of multilingual models - provided your target language was included in the pretraining, **there's a good chance you'll only need to tune and deploy a single model in production instead of running one per language.**

## Using your fine-tuned model

In [27]:
from transformers import pipeline

finetuned_checkpoint = "dmrhimali/xlm-roberta-base-finetuned-marc-en"
classifier = pipeline("text-classification", model=finetuned_checkpoint)

config.json:   0%|          | 0.00/976 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

Device set to use cuda:0


In [28]:
classifier("I loved reading the Hunger Games!")

[{'label': 'great', 'score': 0.7872281074523926}]

In [29]:
classifier("ハンガーゲーム」を読むのが好きだった!")

[{'label': 'great', 'score': 0.5046316385269165}]